# Модуль А.  Парсинг и предобработка данных

### 1.1 Парсинг данных

*Импортируем необходимы библиотеки для дальнейших действий:*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tqdm import notebook
import requests
import re
import os
%matplotlib inline
import glob
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk.stem import wordnet 
from nltk import pos_tag  
from nltk import word_tokenize 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import pairwise_distances 
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression

*Для начало импортируем данные из представленной папки. Они изначально имеют расширение json, так как dataframe лучше рабоет с расширение csv, передем эти файлы в другую папку с данным расширением.Так, как файлов большое количество, импорт будет сделан через корневую дерикторию.*

In [2]:
# modules = glob.glob('data/*.csv')

In [3]:
# data=[]
# for i in modules:
#     data=pd.read_csv(i)

In [4]:
# modules = glob.glob('data\*.json')

In [5]:
# modules

In [6]:
# data_json=[]
# for i in modules:
#     data_json=pd.read_json(i)
#     data_json.to_csv(f'meeting/data.csv')

In [7]:
# data_json=pd.read_json('sample1.json')

In [8]:
# data=pd.read_csv('meeting/data.csv')

In [9]:
# data_json = pd.read_csv('data\sample1.json')

In [10]:
# data_json.to_csv('courses_74.csv')

In [11]:
# data_csv

In [12]:
# data_csv = pd.read_csv('courses_74.csv')

*Теперь необходимо собрать данные из сайта habr.com, в котором хранятся статьи связанные с информационными технологиями, бизнесом и интернетом для того, что бы данные увеличились в размере, тогда прогнозирование будет сделано более качественно.А также поместить их в один dataframe, для того, что бы в дальнейшем было удобнее работать.*

In [13]:
# json_str = '{"Courses":{"r1":"Spark"},"Fee":{"r1":"25000"},"Duration":{"r1":"50 Days"}}'
# df = pd.read_json(json_str)
# df.to_csv('courses.csv')

In [14]:
# df = pd.read_csv("courses.csv")

In [15]:
areas_urls_list = []
date=[]
for i in range(1, 51):
    url = f"https://habr.com/ru/all/page{i}"
    rec=requests.get(url)
    result=rec.content
    soup = BeautifulSoup(result, 'lxml')
    for tag_3 in soup.find_all("span", {'class':'tm-article-datetime-published'}):
        datetime = tag_3.find('time')
        film_list_3 = datetime.get('datetime')
        date.append(film_list_3)
    for tag in soup.find_all("a", {'class':'tm-title__link'}):
        area_info = tag.get("href")
        areas_urls_list.append("https://habr.com"+area_info)

In [16]:
areas_urls_list_2 = []
rating = []
text_states=[]
for item in areas_urls_list:
    url_2 = item
    rec_2=requests.get(url_2)
    result_2=rec_2.content
    soup_2 = BeautifulSoup(result_2, 'lxml')
    for tag_4 in soup_2.find_all("span", {'class':'tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating'}):
        rating.append(tag_4)
   # for tag_6 in soup_2.find_all("div", {'class':'tm-article-presenter'}):
        #dey_2 = tag_6.find('a')
       # url_3 = dey_2.get("title")
       # name.append(url_3)
    for tag_5 in soup_2.find_all("div", {'class':'tm-article-presenter'}):
        url_5=tag_5.find("a")
        url_6 = url_5.get("href")
        areas_urls_list_2.append("https://habr.com"+url_6)
    for tag_7 in soup_2.find_all("div", {'class':'article-formatted-body article-formatted-body article-formatted-body_version-2'}):
        url_7=tag_7.find("p")
        text_states.append(url_7)

In [17]:
name = []
href_add=[]
for i in range(1, 18):
    url_4 = f"https://habr.com/ru/companies/page{i}"
    rec_4=requests.get(url_4)
    result_4=rec_4.content
    soup_4 = BeautifulSoup(result_4, 'lxml')
    for tag_11 in soup_4.find_all("div", {'class':'tm-company-snippet__info'}):
        tag_12=tag_11.find_all("a", {'class':'tm-company-snippet__title'})
        name.append(tag_12)
    for tag_20 in soup_4.find_all("a", {'class':'tm-company-snippet__title'}):
        href = tag_20.get("href")
        href_add.append("https://habr.com"+href)

In [18]:
description=[]
activity=[]
for item in href_add:
    url_8 = item
    rec_8=requests.get(url_8)
    result_8=rec_8.content
    soup_8 = BeautifulSoup(result_8, 'lxml')
    for tag_4 in soup_8.find_all("div", {'class':'tm-company-profile__categories'}):
        tag_5=tag_4.find("span")
        tag_15=tag_5.find("a")
        activity.append(tag_15)
    for tag_7 in soup_8.find_all("span", {'class':'tm-company-profile__content'}):
        description.append(tag_7)

In [19]:
date=pd.DataFrame(date)
rating=pd.DataFrame(rating)
activity=pd.DataFrame(activity)
description=pd.DataFrame(description)
name=pd.DataFrame(name)
text_states=pd.DataFrame(text_states)

C:\Users\blend\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [20]:
data= pd.concat([date, rating, activity, description, name, text_states], axis=1)

In [21]:
data.columns = ['date', 'rating', 'activity', 'description','name', 'text_states']

In [22]:
data

,date,rating,activity,description,name,text_states
0,2023-04-27T23:40:01.000Z,\n 45.8\n,\n Связь и телекоммуникации\n ...,"[Международный облачный провайдер , [RUVDS], ...",[RUVDS.com],"[[В предыдущей статье], я рассказал про проиг..."
1,2023-04-27T22:02:01.000Z,\n 0\n,\n Аппаратное обеспечение\n ...,"[[Selectel], — ведущий в России провайдер обл...",[Selectel],[Захотев однажды научиться разрабатывать ботов...
2,2023-04-27T21:23:59.000Z,\n 7\n,\n Веб-разработка\n,"[[ Timeweb Cloud ], — облачный сервис, сочета...",[Timeweb Cloud],"[Дисплейные методы, как и метод , [гибридóм,],..."
3,2023-04-27T20:22:08.000Z,\n 9\n,\n Мобильные технологии\n ...,"[Строим сервисы, используя силу социальных сет...",[VK],[Горячая тема! Я раньше очень много и тесно ра...
4,2023-04-27T20:10:38.000Z,\n 0\n,\n Консалтинг и поддержка\n ...,"[OTUS – сообщество профессионалов, которые пом...",[OTUS],[В современном мире интернет стал неотъемлемой...
...,...,...,...,...,...,...
993,2023-04-11T08:05:11.000Z,NaN,NaN,NaN,NaN,NaN
994,2023-04-11T07:58:53.000Z,NaN,NaN,NaN,NaN,NaN
995,2023-04-11T07:38:32.000Z,NaN,NaN,NaN,NaN,NaN
996,2023-04-11T07:25:29.000Z,NaN,NaN,NaN,NaN,NaN


*Сохраняем наши данные для дальнейшего использования в других модулях*

In [23]:
data.to_csv("data.csv")

### 1.3 Предварительная обработка текстовых данных  

*В этом разделе необходимо обработать данные, для того, чтобы в дальнейшем обучение модели происходило более качественно, и в дальнейшем можно было разработать API.*

*С помощью функции $info()$ вывожу информацию о данных*

In [24]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 998 entries, 0 to 997
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         998 non-null    object
 1   rating       992 non-null    object
 2   activity     322 non-null    object
 3   description  322 non-null    object
 4   name         322 non-null    object
 5   text_states  825 non-null    object
dtypes: object(6)
memory usage: 342.3 KB


*С помощью функции $describe()$ смотрю статистическую таблицу*

In [25]:
data.describe().T

,count,unique,top,freq
date,998,996,2023-04-21T07:56:23.000Z,2
rating,992,291,\n 6\n,37
activity,322,18,\n Программное обеспечение\n ...,132
description,322,311,[],12
name,322,322,[RUVDS.com],1
text_states,825,808,"[Привет, Хабр!]",7


*Также необхожимо проанализировать, есть ли в данных пропущенные значения.Для этого волспользуюсь функцией **isnull().sum()**.*

In [33]:
data.isnull().sum()

date           0
rating         0
activity       0
description    0
name           0
text_states    0
dtype: int64

*Теперь можно сделать вывод, что большинство значений категориальные, а значит метод обработки данных должен включать обработку текста(токенизацию, лемматизацию, обработку пропущеннных значений, выделение значимых частей речи, а также удаление стоп-слов, пунктуации, спецсимволов). По статистической таблицы видно, что не выбросов, не полхой дисперсии в данных нет, поэтому эти методы обработки применяться не будут.*

*В начале обработаем все значения, которые пропущенны.Так как данных и так не много, воспользуемся стандартными методом.Для категориальных признаков-заменой на моду числа, для количественных-на медианное значение.*

In [30]:
df_non_numeric = data.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

for col in non_numeric_cols:
    missing = data[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:
        print('imputing missing values for: {}'.format(col))
        data['{}_ismissing'.format(col)] = missing
        
        top = data[col].describe()['top']
        data[col] = data[col].fillna(top)

In [31]:
df_numeric = data.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = data[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0: 
        print('imputing missing values for: {}'.format(col))
        data['{}_ismissing'.format(col)] = missing
        med = data[col].median()
        data[col] = data[col].fillna(med)

*Дальше избавимся от ненужных символов в данных и , а также переведем все значения в нижний регистр, для того, что бы потом было удобнее делать токенизацию и , лемматизацию.Также этот метод обработки сделает качественнным дальнейший прогноз.*

In [27]:
def preprocessor(text):
    text=re.sub('<[^>]*>', ' ', text)
    emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text=re.sub('[\W]+', ' ', text.lower())+ ' '.join(emoticons).replace('-', '')
    return text

In [28]:
data=data.astype(str)

In [29]:
for col in data.columns:
    data[col]=data[col].apply(preprocessor)

*Теперь обработаем весь текст в данных методами nltk.Для начало необходимо разделить текст на токены. Дальше исключю стоп-слова из исходного текста, обработаем стриминг текста(удаление окончания слов)-Русский язык обладает богатой морфологической структурой. Слово хороший и хорошая имеют тот же смысл, но разную форму, например, хорошая мебель и хороший стул. Поэтому для машинного обучения (Machine Learning) лучше привести их к одной форме для уменьшения размерности, приведем к начальному значению.* 

**Токенизация**

In [35]:
nltk.download('punkt');

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\blend\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]    

In [37]:
data['name'] = data.apply(lambda x: tokenize(x['name']), axis=1)

In [39]:
data['description'] = data.apply(lambda x: tokenize(x['description']), axis=1)

In [40]:
data['activity'] = data.apply(lambda x: tokenize(x['activity']), axis=1)

In [41]:
data['text_states'] = data.apply(lambda x: tokenize(x['text_states']), axis=1)

**Исключение стоп-слова**

In [48]:
stop_words = stopwords.words('russian')
data['description'] = data['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

AttributeError: 'list' object has no attribute 'split'

**Стриминг**

In [ ]:
stemmer = SnowballStemmer("russian")
data['stemmer'] = data['text'].str.split()
pd.set_option('display.max_colwidth', -1)

**Лемматизация**

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
data['text_lemmatized'] = data.text.apply(lemmatize_text)

In [ ]:
data

### 1.2 Формирование структуры набора данных

*Для выбора признаков, которые следует удалить из данных, я воспользуюсь алгоритмом SBS. Он основан на последовательном удалении признаков из полнопризнакового подмножества.Я взяла конкретно этот алгоритм, потому-что считаю что он один из немногих может обучить текстовые данные хорошо,а также хорошо может работать с данными маленького размера, так как сделан на основе логистической регрессии.*

*Для успешного использования алгоритма, необходимо закодировать категориальные значения.Это позволит улучшить точность выбоора признака.Также необходимо разделить выборку на тестовую и тренировочную.*

In [ ]:
# lbl=LabelEncoder()
# non_nomic=data.select_dtypes(exclude=[np.number])
# non_momic_cols=non_nomic.columns.values
# for col in non_momic_cols:
#     data[col]=lbl.fit_transform(data[col].astype(str))

In [ ]:
# lr = LogisticRegression(penalty='l1', C=0.1)
# lr.fit(X_train_std, y_train)
# print('Training accuracy:', lr.score(X_train_std, y_train))
# print('Test accuracy:', lr.score(X_test_std, y_test))

In [ ]:
# from sklearn.base import clone
# from itertools import combinations
# import numpy as np
# from sklearn.metrics import accuracy_score
# if Version(sklearn_version) < '0.18':
#     from sklearn.cross_validation import train_test_split
# else:
#     from sklearn.model_selection import train_test_split


# class SBS():
#     def __init__(self, estimator, k_features, scoring=accuracy_score,
#                  test_size=0.25, random_state=1):
#         self.scoring = scoring
#         self.estimator = clone(estimator)
#         self.k_features = k_features
#         self.test_size = test_size
#         self.random_state = random_state

#     def fit(self, X, y):
        
#         X_train, X_test, y_train, y_test = \
#             train_test_split(X, y, test_size=self.test_size,
#                              random_state=self.random_state)

#         dim = X_train.shape[1]
#         self.indices_ = tuple(range(dim))
#         self.subsets_ = [self.indices_]
#         score = self._calc_score(X_train, y_train, 
#                                  X_test, y_test, self.indices_)
#         self.scores_ = [score]

#         while dim > self.k_features:
#             scores = []
#             subsets = []

#             for p in combinations(self.indices_, r=dim - 1):
#                 score = self._calc_score(X_train, y_train, 
#                                          X_test, y_test, p)
#                 scores.append(score)
#                 subsets.append(p)

#             best = np.argmax(scores)
#             self.indices_ = subsets[best]
#             self.subsets_.append(self.indices_)
#             dim -= 1

#             self.scores_.append(scores[best])
#         self.k_score_ = self.scores_[-1]

#         return self

#     def transform(self, X):
#         return X[:, self.indices_]

#     def _calc_score(self, X_train, y_train, X_test, y_test, indices):
#         self.estimator.fit(X_train[:, indices], y_train)
#         y_pred = self.estimator.predict(X_test[:, indices])
#         score = self.scoring(y_test, y_pred)
#         return score

### 1.4 Подготовка отчета

*В данном модули были собраны данные с интернет-ресурса **habr.com** и с представленного файл.Произведено предварительное следование данных для того, что-бы понять, какой конкретно метод обработки использовать.Произведена предобработка данных для дальнейшего обучения модели и разработки api.Также были отобраны признаки, которые больше всего зависят от целевой переменной, а также предварительная обрабоотка текста, что бы разделение на кластеры было более качественно сделано.По итогу файл с предобработанными данными был загружен, для дальнейшего использования.*